# Display images 

In [ ]:
from IPython.display import Image, display, clear_output
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams
import os
import json
import time
import copy
%matplotlib inline


In [ ]:
path = "/root/flink-pravega-demo/out.csv"

rcParams['figure.figsize'] = 16 ,8

In [ ]:
import csv
from datetime import datetime


img1 = None
img2 = None
fail_behavior1 = []
fail_behavior2 = []
img_location1 = ""
img_locatopn2 = ""
while(True):
    fail_count = {"driver1":{"both_hands_leaving_wheel":0,"eyes_closed":0,"not_buckling_up":0,"smoke":0,"not_facing_front":0,"cellphone":0,"yawning":0,"head_lowered":0}, 
                  "driver2":{"both_hands_leaving_wheel":0,"eyes_closed":0,"not_buckling_up":0,"smoke":0,"not_facing_front":0,"cellphone":0,"yawning":0,"head_lowered":0}}
    if not os.path.exists(path):
        print("No data availiable!")
        time.sleep(1)
        continue
    with open(path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            plt.ion()
            fig, ax = plt.subplots(2,2,figsize = (20, 15))
#             ax[0][0].axis("off")
#             ax[1][0].axis("off")
            ax[1][1].axis("off")
            ax[0][1].axis("off")
            fail_behavior = []
            img_raw = row[1]
            result = json.loads(row[2])
            if result.get('person_info'):
                for key in result['person_info'][0]['attributes']:
                    if result['person_info'][0]['attributes'][key]['score'] > result['person_info'][0]['attributes'][key]['threshold']:
                        fail_behavior.append(key)
                        if fail_count[row[0]].get(key):
                            fail_count[row[0]][key] +=1 
                        else:
                            fail_count[row[0]][key] =1
#             ax[0].clear
#             ax[1].clear
            if row[0] == "driver1":
                img1 = mpimg.imread(img_raw)
                img_location1 = img_raw

            else:
                img2 = mpimg.imread(img_raw)
                img_location2 = img_raw

            if img1 is not None:
                ax[0][0].imshow(img1)
                ax[0][0].set_title("Driver1")
                ax[0][0].set_xlabel(img_location1.split("/")[-1])
            if img2 is not None:
                ax[1][0].imshow(img2)
                ax[1][0].set_title("Driver2")
                ax[1][0].set_xlabel(img_location2.split("/")[-1])
            if not fail_behavior:
                fail_behavior.append("Good")
            if row[0] == "driver1":
                fail_behavior1 = copy.deepcopy(fail_behavior)
            else:
                fail_behavior2 = copy.deepcopy(fail_behavior)

#                 if row[0] == "driver1":
#                     ax[0].title.set_text("Good")
#                 else:
#                     ax[1].title.set_text("Good")
#             ax[0].title.set_text("|".join(fail_behavior1))
#             ax[1].title.set_text("|".join(fail_behavior2))
            output1 = "----------------------------------\nCurrent illigal behavior:\n----------------------------------\n"
            output2 = "----------------------------------\nCurrent illigal behavior:\n----------------------------------\n"
            output3 = "-----------------------------------\nTotal illigal behavior:\n-----------------------------------\n"
            output4 = "-----------------------------------\nTotal illigal behavior:\n-----------------------------------\n"
            for key in fail_count:
                for item in fail_count[key]:
                    if key == "driver1":
                        output3 = output3 + item + ":" + str(fail_count[key][item])+"\n"
                    if key == "driver2":
                        output4 = output4 + item + ":" + str(fail_count[key][item])+"\n"
            output1 = output1 + '\n'.join(fail_behavior1)
            output2 = output2 + '\n'.join(fail_behavior2)
            ax[0][1].text(0, 0, output1+"\n\n"+output3+"\n",fontsize=14)
            ax[1][1].text(0, 0, output2+"\n\n"+output4+"\n",fontsize=14) 
            plt.rcParams["axes.edgecolor"] = "0.15"
            plt.rcParams["axes.linewidth"] = 1.25
                    #output = output + key+":"+item+"["+str(fail_count[key][item])+"]"
            #plt.show()
#             fig.canvas.draw()
            plt.rcParams["figure.figsize"] = (100, 1000)
            plt.show()
#             print(output)
#             time.sleep(1)
            clear_output(wait=True)
    time.sleep(3)
